In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../')

import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import itertools

from config import (DATA_PATH, DATE_COL, SPLIT_COL, TARGET_COL, L, WARMUP,           
    SCENARIOS, H_DEFAULT, BETA_GRID_TREE, ETA_GRID_TREE)

from helpers import (add_leadtime_target_noleak, build_group_cache, pooled_empirical_from_train,
    tune_beta_eta_fast_core, _simulate_rq_multi, aggregate_matrix)

RANDOM_SEED: int = 123
np.random.seed(RANDOM_SEED)

def tune_beta_eta_fast(leaf_ids_val, leaf_ids_train, y_train, betas, K_order, p, h, L,                   
    warmup, eta_grid, service_min=0.95, fill_min=0.95):
    return tune_beta_eta_fast_core(leaf_ids_val=leaf_ids_val, leaf_ids_train=leaf_ids_train, y_train=y_train,                 
        betas=np.asarray(betas, float), K_order=K_order, p=p, h=h, L=L, warmup=warmup,                 
        eta_grid=np.asarray(eta_grid, float), service_min=service_min, fill_min=fill_min, val_dem_cat=val_dem_cat,          
        val_pos_cat=val_pos_cat, val_starts=val_starts)

In [2]:
# reading the dataset
df = pd.read_csv(DATA_PATH).copy()

# constructing a composite sku key if it is not already present in the dataset
if "sku" not in df.columns:
    df["sku"] = df["store_id"].astype(str) + "_" + df["item_id"].astype(str)

# parsing the date column 
df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")

# normalizing the split labels to a consistent lowercase format with no stray spaces
df[SPLIT_COL] = df[SPLIT_COL].astype(str).str.lower().str.strip()

# ensuring there is a demand column; if not present, mapping from sales and clipping to nonnegative
if "demand" not in df.columns:
    df["demand"] = df["sales"].astype(float).clip(lower=0.0)

# sorting the entire table by (sku,date) to make group-by operations deterministic
df = df.sort_values(["sku", DATE_COL]).reset_index(drop=True)

# building the leak-safe lead-time target D_L strictly under same-split windows
dl = add_leadtime_target_noleak(df[["sku", DATE_COL, "demand", SPLIT_COL]], L=L, split_col=SPLIT_COL)

# merging the constructed D_L back to the main frame in a one-to-one fashion
df = df.merge(dl[["sku", DATE_COL, TARGET_COL]], on=["sku", DATE_COL], how="left", validate="one_to_one")

# creating a boolean mask to indicate which rows actually have a finite D_L target
has_label = ~df[TARGET_COL].isna()

# building split masks
is_tr = df[SPLIT_COL].eq("train") & has_label
is_va = df[SPLIT_COL].isin(["val", "validation"]) & has_label
is_te = df[SPLIT_COL].eq("test")

# defining columns that must never be used as model inputs (keys, labels, raw demand)
drop_cols = {"sales", "demand", TARGET_COL, "date", "sku", SPLIT_COL, 'dept_roll_mean_28','cumulative_sales'}

# selecting only the numeric features
feature_cols = [
    c for c in df.columns
    if (c not in drop_cols) and pd.api.types.is_numeric_dtype(df[c])
]
print(feature_cols)

# extracting TRAIN feature matrix (aligned with the TRAIN mask)
X_tr = df.loc[is_tr, feature_cols].reset_index(drop=True)

# extracting TRAIN target vector as float64 numpy array
y_tr = df.loc[is_tr, TARGET_COL].to_numpy(np.float64)

# extracting VAL feature matrix (aligned with the VAL mask)
X_va = df.loc[is_va, feature_cols].reset_index(drop=True)

# extracting TEST feature matrix (aligned with the TEST mask)
X_te = df.loc[is_te, feature_cols].reset_index(drop=True)

# collecting the VAL key frame for later merging and output 
val_keys = df.loc[is_va, ["sku", DATE_COL]].reset_index(drop=True)

# collecting the TEST key frame for later merging and output 
test_keys = df.loc[is_te, ["sku", DATE_COL]].reset_index(drop=True)

# preparing a VAL demand frame (sku,date,demand) to drive the simulator
val_sim_df = (df.loc[df[SPLIT_COL].isin(["val", "validation"]), ["sku", DATE_COL, "demand"]]
      .sort_values(["sku", DATE_COL]).reset_index(drop=True))

# preparing a TEST demand frame (sku,date,demand) to drive the simulator
test_sim_df = (df.loc[df[SPLIT_COL].eq("test"), ["sku", DATE_COL, "demand"]]
      .sort_values(["sku", DATE_COL]).reset_index(drop=True))

# building the VAL caches that the simulator and tuner expect 
val_dem_cat, val_pos_cat, val_starts = build_group_cache(val_keys, val_sim_df)

# building the TEST caches in the same format to avoid re-joining inside loops
test_dem_cat, test_pos_cat, test_starts = build_group_cache(test_keys, test_sim_df)

['wday', 'year', 'sell_price', 'is_event', 'snap', 'day', 'has_event2', 'lag_sales_1', 'lag_sales_7', 'lag_sales_14', 'lag_sales_28', 'lag_sales_56', 'lag_sales_365', 'roll_mean_7', 'roll_std_7', 'roll_mean_28', 'roll_std_28', 'roll_mean_56', 'roll_std_56', 'roll_mean_365', 'roll_std_365', 'lag_price_1', 'roll_price_7', 'price_change_1', 'days_since_start', 'event_name_1_Chanukah End', 'event_name_1_Christmas', 'event_name_1_Cinco De Mayo', 'event_name_1_ColumbusDay', 'event_name_1_Easter', 'event_name_1_Eid al-Fitr', 'event_name_1_EidAlAdha', "event_name_1_Father's day", 'event_name_1_Halloween', 'event_name_1_IndependenceDay', 'event_name_1_LaborDay', 'event_name_1_LentStart', 'event_name_1_LentWeek2', 'event_name_1_MartinLutherKingDay', 'event_name_1_MemorialDay', "event_name_1_Mother's day", 'event_name_1_NBAFinalsEnd', 'event_name_1_NBAFinalsStart', 'event_name_1_NewYear', 'event_name_1_OrthodoxChristmas', 'event_name_1_OrthodoxEaster', 'event_name_1_Pesach End', 'event_name_1_Pre

In [3]:
# initializing a small, regularized regression tree for clustering
tree = DecisionTreeRegressor(
    max_depth=7,                
    min_samples_leaf=800,       
    random_state=RANDOM_SEED     
)

# fitting the tree on TRAIN only to avoid any leakage from VAL/TEST
tree.fit(X_tr, y_tr)

# computing the TRAIN leaf ids which will be used as the source pools for quantiles/means
leaf_tr = tree.apply(X_tr).astype(np.int64)

# computing the VAL leaf ids which will be used for tuning (beta,eta)
leaf_va = tree.apply(X_va).astype(np.int64)

# computing the TEST leaf ids which will be used for inference of (r,Q)
leaf_te = tree.apply(X_te).astype(np.int64)

all_results = []    
all_rQ_rows = []   

# iterating over the scenarios 
for sc in SCENARIOS:
    Kc, pc, hc = float(sc["K"]), float(sc["p"]), float(sc["h"])
    print(f"\n[M5 Cluster+Regressor] scenario K={Kc}, p={pc}, h={hc}")

    # computing pooled VAL quantiles/means by looking up each VAL row’s TRAIN leaf
    q_val, mu_val = pooled_empirical_from_train(leaf_va, leaf_tr, y_tr, BETA_GRID_TREE)
    # precomputing EOQ (without eta) once for VAL because it does not depend on beta
    mu_rate_val = np.maximum(mu_val, 0.0) / float(L)
    Q_eoq_val   = np.sqrt((2.0 * Kc * mu_rate_val) / max(hc, 1e-12))

    best_feas = None
    best_any  = None
    best_payload = None

    # looping over beta candidates to test different r-levels
    for b_idx, beta in enumerate(BETA_GRID_TREE):
        # forming the reorder point vector for VAL by taking the beta-quantile from pooled stats
        r_val = q_val[:, b_idx]
        # expanding r to the per-day level using the prebuilt (sku,date) → position mapping
        r_cat = r_val[val_pos_cat]

        # looping over eta candidates to scale the economic order quantities into the final Q values
        for eta in ETA_GRID_TREE:
            # building the full Q vector on VAL by scaling EOQ and respecting the lower bound of 1
            Q_val = np.maximum(Q_eoq_val * eta, 1.0)
            # expanding Q to the per-day level (same mapping as r)
            q_cat = Q_val[val_pos_cat]

            # running the simulator on VAL for this (beta, eta) pair
            out_mat = _simulate_rq_multi(val_dem_cat, r_cat, q_cat, val_starts, L, Kc, hc, pc, WARMUP)
            agg = aggregate_matrix(out_mat)

            # extracting the service and fill metrics along with total and holding cost (used for tie-breaks)
            svc, fill = agg["service_level"], agg["beta_fill_rate"]
            tot, hold = agg["total_cost"], agg["holding_cost"]

            # building a lexicographic key for feasible candidates (svc/fill ≥ thresholds)
            if (svc >= 0.95) and (fill >= 0.95):
                key = (tot, hold, -svc, -fill, float(beta), float(eta))
                # updating the best feasible if this candidate is strictly better
                if (best_feas is None) or (key < best_feas):
                    best_feas = key
                    best_payload = dict(beta=float(beta), eta_Q=float(eta),
                                        total_mean=float(tot), service_mean=float(svc), fill_mean=float(fill))

            # also tracking the best overall candidate by minimizing shortfall first (in case nothing is feasible)
            shortfall = max(0.0, 0.95 - svc) + max(0.0, 0.95 - fill)
            key_any = (shortfall, tot, -svc, -fill, float(beta), float(eta))
            if (best_any is None) or (key_any < best_any):
                best_any = key_any
                if best_payload is None:  
                    best_payload = dict(beta=float(beta), eta_Q=float(eta),
                                        total_mean=float(tot), service_mean=float(svc), fill_mean=float(fill))

    # deciding which candidate to adopt: feasible if we found one; otherwise the best "least bad" option
    chosen = best_feas if best_feas is not None else best_any
    assert chosen is not None, "tuning produced no candidate; check inputs"
    beta_star = float(chosen[4])
    eta_star  = float(chosen[5])
    print(f"  tuned (VAL): beta={beta_star:.3f}, eta_Q={eta_star:.2f} | "
          f"svc={best_payload['service_mean']:.3f}, fill={best_payload['fill_mean']:.3f}, "
          f"total={best_payload['total_mean']:.2f}")

    # obtaining TRAIN-time r at the tuned beta by looking up each TRAIN row’s own leaf (constant within leaf)
    q_tr_leaf, mu_tr_leaf = pooled_empirical_from_train(leaf_tr, leaf_tr, y_tr, [beta_star])
    # squeezing q_tr to shape (N_train,) because pooled_empirical returns 2D for quantiles
    r_tr_per_row = q_tr_leaf[:, 0]
    # computing per-row EOQ on TRAIN using the leaf mean, then scaling by eta
    mu_rate_tr = np.maximum(mu_tr_leaf, 0.0) / float(L)
    Q_tr_per_row = np.maximum(np.sqrt((2.0 * Kc * mu_rate_tr) / max(hc, 1e-12)) * eta_star, 1.0)

    # stacking the two labels into a multi-output Y matrix of shape (N_train, 2) in the order [r, Q]
    Y_tr_rq = np.column_stack([r_tr_per_row.astype(np.float64), Q_tr_per_row.astype(np.float64)])

    # instantiating a compact random forest
    rf_rq = RandomForestRegressor(
        n_estimators=300,         
        max_depth=12,           
        min_samples_leaf=50,       
        n_jobs=-1,                 
        random_state=RANDOM_SEED   
    )
    # training the regressor on TRAIN features with the constructed (r,Q) labels
    rf_rq.fit(X_tr, Y_tr_rq)

    # running the regressor on TEST features to obtain a (n_test × 2) matrix of predictions
    Y_te_hat = rf_rq.predict(X_te)
    # slicing the two columns into separate arrays and enforcing the lower bound on Q
    r_te_hat = Y_te_hat[:, 0].astype(np.float64)
    Q_te_hat = np.maximum(Y_te_hat[:, 1], 1.0).astype(np.float64)

    # mapping the row-level predictions to per-day arrays using the cached (sku,date) → index mapping
    r_cat = r_te_hat[test_pos_cat]
    q_cat = Q_te_hat[test_pos_cat]

    # running the batched TEST simulation for this scenario with the learned (r, Q) paths
    out_mat = _simulate_rq_multi(test_dem_cat, r_cat, q_cat, test_starts, L, Kc, hc, pc, WARMUP)
    agg = aggregate_matrix(out_mat)

    agg.update(dict(model="tree_cluster_regressor", K=int(Kc), p=int(pc), h=float(hc),
                    L=int(L), beta=beta_star, eta_Q=eta_star, n_skus=out_mat.shape[0]))
    all_results.append(agg)

    rQ_te = test_keys.copy()
    rQ_te["r"]    = r_te_hat
    rQ_te["Q"]    = Q_te_hat
    rQ_te["K"]    = int(Kc); rQ_te["p"] = int(pc); rQ_te["h"] = float(hc); rQ_te["L"] = int(L)
    rQ_te["beta"] = beta_star; rQ_te["eta_Q"] = eta_star
    rQ_te["model"]= "tree_cluster_regressor"
    all_rQ_rows.append(rQ_te)

results_df = pd.DataFrame(all_results)
if not results_df.empty:
    scenario_cols = ["K","p","h","L","model","beta","eta_Q","n_skus"]
    other_cols    = [c for c in results_df.columns if c not in scenario_cols]
    results_df    = results_df[scenario_cols + other_cols].sort_values(["K","p"]).reset_index(drop=True)
    results_df.to_csv("m5_cluster_regressor_results.csv", index=False)

rQ_out = pd.concat(all_rQ_rows, ignore_index=True) if all_rQ_rows else pd.DataFrame()
if not rQ_out.empty:
    rQ_out.to_csv("m5_cluster_regressor_rQ.csv", index=False)

print("Cluster-based + Regressor results summary")
print(results_df)


[M5 Cluster+Regressor] scenario K=10.0, p=1.0, h=0.1
  tuned (VAL): beta=0.800, eta_Q=1.10 | svc=0.969, fill=0.959, total=974.09

[M5 Cluster+Regressor] scenario K=10.0, p=3.0, h=0.1
  tuned (VAL): beta=0.800, eta_Q=1.20 | svc=0.970, fill=0.961, total=1232.70

[M5 Cluster+Regressor] scenario K=10.0, p=9.0, h=0.1
  tuned (VAL): beta=0.940, eta_Q=1.20 | svc=0.988, fill=0.981, total=1737.53

[M5 Cluster+Regressor] scenario K=50.0, p=1.0, h=0.1
  tuned (VAL): beta=0.600, eta_Q=0.90 | svc=0.961, fill=0.953, total=1521.74

[M5 Cluster+Regressor] scenario K=50.0, p=3.0, h=0.1
  tuned (VAL): beta=0.600, eta_Q=1.20 | svc=0.970, fill=0.963, total=1763.06

[M5 Cluster+Regressor] scenario K=50.0, p=9.0, h=0.1
  tuned (VAL): beta=0.880, eta_Q=1.10 | svc=0.990, fill=0.985, total=2212.40

[M5 Cluster+Regressor] scenario K=100.0, p=1.0, h=0.1
  tuned (VAL): beta=0.600, eta_Q=0.90 | svc=0.971, fill=0.964, total=1995.89

[M5 Cluster+Regressor] scenario K=100.0, p=3.0, h=0.1
  tuned (VAL): beta=0.600, e

In [4]:
# defining a compact grid for depth and minimum leaf size to keep HPO fast
DEPTHS    = [7, 10, 15]
# choosing a few leaf size thresholds to ensure decent pooling in leaves
MIN_LEAFS = [400, 800, 1200]

best_score   = None
best_params  = None
best_tree    = None
best_leaf_tr = None
best_leaf_va = None

# scanning the small grid in a deterministic order
for md, ml in itertools.product(DEPTHS, MIN_LEAFS):
    # creating a candidate regression tree with the current (depth, min leaf)
    cand = DecisionTreeRegressor(
        max_depth=md,            
        min_samples_leaf=ml,          
        random_state=RANDOM_SEED      
    )
    # fitting the candidate on TRAIN only to keep validation honest
    cand.fit(X_tr, y_tr)

    # computing TRAIN leaves for pooling empirical stats
    ltr = cand.apply(X_tr).astype(np.int64)
    # computing VAL leaves for the tuning step
    lva = cand.apply(X_va).astype(np.int64)

    shortfalls, totals = [], []

    # iterating across all scenarios 
    for sc in SCENARIOS:
        Kc, pc, hc = float(sc["K"]), float(sc["p"]), float(sc["h"])

        # tuning (beta,eta) on VAL with the current candidate tree
        best = tune_beta_eta_fast(
            leaf_ids_val=lva,               
            leaf_ids_train=ltr,           
            y_train=y_tr,                 
            betas=BETA_GRID_TREE,         
            K_order=Kc, p=pc, h=hc,         
            L=L, warmup=WARMUP,          
            eta_grid=ETA_GRID_TREE,         
            service_min=0.95, fill_min=0.95 
        )

        # extracting the validation metrics that define the score
        svc, fill, tot = best["service_mean"], best["fill_mean"], best["total_mean"]

        # computing a combined shortfall relative to the service/fill targets
        shortfall = max(0.0, 0.95 - svc) + max(0.0, 0.95 - fill)

        # recording the shortfall for this scenario
        shortfalls.append(shortfall)
        # recording the total cost for this scenario
        totals.append(tot)

    # computing the lexicographic score as (mean shortfall, mean total cost)
    score = (float(np.mean(shortfalls)), float(np.mean(totals)))

    # updating the incumbent if this candidate scores strictly better
    if (best_score is None) or (score < best_score):
        best_score   = score                         # storing the new best score
        best_params  = dict(max_depth=md, min_samples_leaf=ml)  # storing params
        best_tree    = cand                          # keeping the fitted tree
        best_leaf_tr = ltr                           # caching TRAIN leaves
        best_leaf_va = lva                           # caching VAL leaves

print("[HPO] Best tree params:", best_params, "score=", best_score)

[HPO] Best tree params: {'max_depth': 10, 'min_samples_leaf': 400} score= (0.0, 1772.1624674862426)


In [5]:
# computing TEST leaf ids for the selected tree so we can map TEST rows into its partition
best_leaf_te = best_tree.apply(X_te).astype(np.int64)

all_results_hpo = []     
all_rQ_rows_hpo = []    

# iterating across the scenarios 
for sc in SCENARIOS:
    # unpacking the scenario parameters (order cost, backorder cost, holding cost)
    Kc, pc, hc = float(sc["K"]), float(sc["p"]), float(sc["h"])
    print(f"\n[M5 Cluster+Regressor (HPO tree)] scenario K={Kc}, p={pc}, h={hc}")

    # computing pooled VAL quantiles/means using best_tree leaves (VAL rows lookup TRAIN leaves of best_tree)
    q_va_best, mu_va_best = pooled_empirical_from_train(best_leaf_va, best_leaf_tr, y_tr, BETA_GRID_TREE)
    # precomputing EOQ backbone for VAL (depends on mu, K, h, L) and postpone scaling by eta
    mu_rate_va = np.maximum(mu_va_best, 0.0) / float(L)
    Q_eoq_va   = np.sqrt((2.0 * Kc * mu_rate_va) / max(hc, 1e-12))

    best_feas = None
    best_any  = None
    best_payload = None

    # scanning the beta grid first (discrete quantile choices)
    for b_idx, beta in enumerate(BETA_GRID_TREE):
        # forming the reorder point vector on VAL from pooled quantiles
        r_val = q_va_best[:, b_idx]
        # mapping the row-level r into the per-day layout via the cached indices
        r_cat = r_val[val_pos_cat]

        # scanning eta to determine the lot-sizing scale factor
        for eta in ETA_GRID_TREE:
            # building the final Q on VAL and enforcing Q ≥ 1
            Q_val = np.maximum(Q_eoq_va * eta, 1.0)
            # mapping to per-day layout for the simulator
            q_cat = Q_val[val_pos_cat]

            # simulating the full VAL panel for this candidate
            out_mat = _simulate_rq_multi(val_dem_cat, r_cat, q_cat, val_starts, L, Kc, hc, pc, WARMUP)
            agg = aggregate_matrix(out_mat)

            # extracting metrics used for constraints and tie-breaking
            svc, fill = agg["service_level"], agg["beta_fill_rate"]
            tot, hold = agg["total_cost"], agg["holding_cost"]

            # updating the best feasible choice (svc/fill ≥ 0.95)
            if (svc >= 0.95) and (fill >= 0.95):
                key = (tot, hold, -svc, -fill, float(beta), float(eta))
                if (best_feas is None) or (key < best_feas):
                    best_feas = key
                    best_payload = dict(beta=float(beta), eta_Q=float(eta),
                                        total_mean=float(tot), service_mean=float(svc), fill_mean=float(fill))

            # updating the best overall fallback by minimizing shortfall first
            shortfall = max(0.0, 0.95 - svc) + max(0.0, 0.95 - fill)
            key_any = (shortfall, tot, -svc, -fill, float(beta), float(eta))
            if (best_any is None) or (key_any < best_any):
                best_any = key_any
                if best_payload is None:
                    best_payload = dict(beta=float(beta), eta_Q=float(eta),
                                        total_mean=float(tot), service_mean=float(svc), fill_mean=float(fill))

    # extracting the chosen (beta, eta) pair from the trackers
    chosen = best_feas if best_feas is not None else best_any
    assert chosen is not None, "tuning produced no candidate; check inputs"
    beta_star = float(chosen[4])
    eta_star  = float(chosen[5])
    print(f"  tuned (VAL, HPO tree): beta={beta_star:.3f}, eta_Q={eta_star:.2f} | "
          f"svc={best_payload['service_mean']:.3f}, fill={best_payload['fill_mean']:.3f}, "
          f"total={best_payload['total_mean']:.2f}")

    # obtaining r on TRAIN at the tuned beta using best_tree leaf assignments
    q_tr_best, mu_tr_best = pooled_empirical_from_train(best_leaf_tr, best_leaf_tr, y_tr, [beta_star])
    # converting the quantile matrix (N×1) to a simple vector for convenience
    r_tr_per_row = q_tr_best[:, 0]
    # computing TRAIN EOQ and scaling by eta to get final Q labels
    mu_rate_tr = np.maximum(mu_tr_best, 0.0) / float(L)
    Q_tr_per_row = np.maximum(np.sqrt((2.0 * Kc * mu_rate_tr) / max(hc, 1e-12)) * eta_star, 1.0)

    # stacking the two targets into a (N_train × 2) label matrix in [r, Q] order
    Y_tr_rq = np.column_stack([r_tr_per_row.astype(np.float64), Q_tr_per_row.astype(np.float64)])

    # creating a compact random forest for the (r, Q) mapping
    rf_rq = RandomForestRegressor(
        n_estimators=300,
        max_depth=12,
        min_samples_leaf=50,
        n_jobs=-1,
        random_state=RANDOM_SEED
    )
    # fitting the regressor with TRAIN features and labels
    rf_rq.fit(X_tr, Y_tr_rq)

    # running predictions for TEST rows to get a (n_test × 2) matrix
    Y_te_hat = rf_rq.predict(X_te)
    # splitting into separate vectors and enforcing Q ≥ 1
    r_te_hat = Y_te_hat[:, 0].astype(np.float64)
    Q_te_hat = np.maximum(Y_te_hat[:, 1], 1.0).astype(np.float64)

    # mapping row-level predictions to the per-day simulator layout using cached indices
    r_cat = r_te_hat[test_pos_cat]
    q_cat = Q_te_hat[test_pos_cat]

    # simulating the inventory system on TEST with the learned (r, Q) trajectories
    out_mat = _simulate_rq_multi(test_dem_cat, r_cat, q_cat, test_starts, L, Kc, hc, pc, WARMUP)
    agg = aggregate_matrix(out_mat)

    agg.update(dict(model="tree_cluster_regressor_hpo", K=int(Kc), p=int(pc), h=float(hc),
                    L=int(L), beta=beta_star, eta_Q=eta_star, n_skus=out_mat.shape[0]))
    all_results_hpo.append(agg)

    rQ_te = test_keys.copy()
    rQ_te["r"]    = r_te_hat
    rQ_te["Q"]    = Q_te_hat
    rQ_te["K"]    = int(Kc); rQ_te["p"] = int(pc); rQ_te["h"] = float(hc); rQ_te["L"] = int(L)
    rQ_te["beta"] = beta_star; rQ_te["eta_Q"] = eta_star
    rQ_te["model"]= "tree_cluster_regressor_hpo"
    all_rQ_rows_hpo.append(rQ_te)

results_hpo_df = pd.DataFrame(all_results_hpo)
if not results_hpo_df.empty:
    scenario_cols = ["K","p","h","L","model","beta","eta_Q","n_skus"]
    other_cols    = [c for c in results_hpo_df.columns if c not in scenario_cols]
    results_hpo_df = results_hpo_df[scenario_cols + other_cols].sort_values(["K","p"]).reset_index(drop=True)
    results_hpo_df.to_csv("m5_cluster_regressor_results_tuned.csv", index=False)

rQ_hpo_out = pd.concat(all_rQ_rows_hpo, ignore_index=True) if all_rQ_rows_hpo else pd.DataFrame()
if not rQ_hpo_out.empty:
    rQ_hpo_out.to_csv("m5_cluster_regressor_rQ_tuned.csv", index=False)

print("Cluster-based + Regressor tuned results summary")
print(results_hpo_df)


[M5 Cluster+Regressor (HPO tree)] scenario K=10.0, p=1.0, h=0.1
  tuned (VAL, HPO tree): beta=0.800, eta_Q=1.00 | svc=0.964, fill=0.953, total=942.13

[M5 Cluster+Regressor (HPO tree)] scenario K=10.0, p=3.0, h=0.1
  tuned (VAL, HPO tree): beta=0.800, eta_Q=1.20 | svc=0.969, fill=0.961, total=1182.51

[M5 Cluster+Regressor (HPO tree)] scenario K=10.0, p=9.0, h=0.1
  tuned (VAL, HPO tree): beta=0.940, eta_Q=1.20 | svc=0.988, fill=0.981, total=1667.20

[M5 Cluster+Regressor (HPO tree)] scenario K=50.0, p=1.0, h=0.1
  tuned (VAL, HPO tree): beta=0.600, eta_Q=1.00 | svc=0.965, fill=0.957, total=1517.53

[M5 Cluster+Regressor (HPO tree)] scenario K=50.0, p=3.0, h=0.1
  tuned (VAL, HPO tree): beta=0.600, eta_Q=1.10 | svc=0.969, fill=0.961, total=1753.53

[M5 Cluster+Regressor (HPO tree)] scenario K=50.0, p=9.0, h=0.1
  tuned (VAL, HPO tree): beta=0.880, eta_Q=1.10 | svc=0.990, fill=0.984, total=2099.54

[M5 Cluster+Regressor (HPO tree)] scenario K=100.0, p=1.0, h=0.1
  tuned (VAL, HPO tree)